In [4]:
from gensim.models import FastText as FT

model = FT.load('../../data/word_vector/KT_model')
print(len(model.wv.vocab))

18754


In [2]:
print(model.wv.most_similar('映画'))

[('作品', 0.9072827696800232), ('出演', 0.8487749695777893), ('ドラマ', 0.8376592993736267), ('監督', 0.8273282647132874), ('也監督', 0.8245197534561157), ('加恋', 0.7967026829719543), ('舞台', 0.7932311296463013), ('異例', 0.7928273677825928), ('一挙', 0.7903098464012146), ('ラブストーリー', 0.79002445936203)]


In [ ]:
from gensim.models import KeyedVectors
word_vector = model.wv
word_vector.save('sample.kv')

In [9]:
del reload_wv
reload_wv = KeyedVectors.load('sample.kv', mmap='r')

In [11]:
print(reload_wv.most_similar('出演'))

[('監督', 0.8709227442741394), ('異例', 0.8695576786994934), ('也監督', 0.8675853610038757), ('加恋', 0.8592053651809692), ('部門', 0.8523834347724915), ('会', 0.8516529202461243), ('キャスト', 0.8512829542160034), ('映画', 0.8487749695777893), ('会場', 0.8481562733650208), ('賞', 0.8463965654373169)]


In [7]:
model.wv.save_word2vec_format('temp.vec')

In [1]:
from torchtext.vocab import Vectors

torch_model = Vectors(name='temp.vec')

In [2]:
print(torch_model.dim)
print(torch_model.itos[50:60])

150
['利用', '者', 'ソフトウェア', '搭載', '時', 'もの', 'モデル', 'ため', '氏', 'いい']


In [23]:
print(torch_model.get_vecs_by_tokens('利用'))

tensor([ 0.3899, -0.3898, -0.6604, -1.2381, -1.0817,  1.4503, -0.1950, -0.0019,
         0.6717, -0.7439,  0.0457, -0.4733,  0.0386, -0.6868,  0.0529, -0.5247,
        -0.4720, -1.2356,  0.4112,  0.2987, -1.0640,  0.8272,  0.2465, -0.3243,
         1.0520,  1.2802,  0.2264, -0.4844, -0.5899, -1.0869,  1.7414, -0.8604,
        -0.9312, -1.1254,  0.8184, -0.5523, -1.0133,  1.1945, -0.6693,  1.1393,
        -0.3109,  0.2209, -1.0506, -0.4818,  0.2712,  0.6679, -1.0131,  0.8487,
         0.1403, -1.2270,  0.3953,  0.4735, -0.5317,  0.1851,  0.0170,  1.5596,
         0.1738,  0.1890,  0.3338,  1.1207, -0.3757, -0.7718, -0.6664, -0.8079,
        -0.0148, -0.3435,  0.7334,  0.5863,  0.1468,  0.8084, -0.0796,  0.9579,
        -0.9285, -0.8729,  1.0437,  0.5003,  0.8108,  0.4469, -0.5699, -0.8651,
        -0.3200, -0.5215, -1.0067,  0.2240, -0.3499, -0.7361,  1.1988, -0.0346,
         1.3119, -1.5588, -0.3179,  0.7576,  0.0078, -0.3217,  0.9052, -1.0159,
        -0.2114,  0.1833,  0.8502, -1.44

In [45]:
import torch.nn.functional as F
target_vec = torch_model.get_vecs_by_tokens('映画')
entry_vec = torch_model.get_vecs_by_tokens('テレビ')
score_1 = F.cosine_similarity(target_vec, entry_vec, dim=0)
print(score_1)
entry_vec = torch_model.get_vecs_by_tokens('クジ')
score_2 = F.cosine_similarity(target_vec, entry_vec, dim=0)
print(score_2)

print(score_1 > score_2)
print(score_1 < score_2)

tensor(0.6465)
tensor(-0.0245)
tensor(True)
tensor(False)


In [55]:
import torch
import torch.nn.functional as F

def get_similarity(model, word):

    target_vec = torch_model.get_vecs_by_tokens(word)

    for i, vocab in enumerate(model.itos):
        if vocab != word:
            entry_vec = torch_model.get_vecs_by_tokens(vocab)
            score = F.cosine_similarity(target_vec, entry_vec, dim=0)
            if i == 0:
                best = score
                match = vocab
            if score > best:
                best = score
                match = vocab

    return best, match

In [61]:
best_score, match_word = get_similarity(torch_model, 'コンピュータ')
print('score: {}, word: {}'.format(best_score, match_word))

score: 0.9568933248519897, word: フィッシャー
